In [1]:
##### This code is from Updated Testing Reddit - No Con- bias (Fictitious Play)-01092022
##### This code replace the big real datanetwork with small sythetic network
import scipy
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import time
import random
from scipy.stats import beta
import pandas as pd
import copy
%matplotlib inline
# %run Check_Derivation_of_Two_Opinions.ipynb
#%run pure_strategy_selection.ipynb  #include simple selection algorithm
import scipy.io
import collections
import sys
from itertools import product
import scipy.optimize
import csv
import os
import os.path
from math import comb

save_path = os.getcwd()

    # Fixed initial condition + memeory = 50

## Mathmatic Functions

In [2]:
# centers the opinion vector around 0\n",
def mean_center(op, n):
    ones = np.ones((n, 1))
    x = op - (np.dot(np.transpose(op),ones)/n) * ones
    return x

# compute number of edges, m\n
def num_edges(L, n):
    m = 0
    for i in range(n):
        for j in range(n):
            if i > j and L[i,j] < 0:
                m += 1
    return m

# Compute number of possible actions
def len_actions(k, n):
    # create all combination of K opinions
    max_option = [0, 1]

    k_opinions =list(product(max_option, repeat=k))  # All k opinion combinations
    len_kops = len(k_opinions) # - number of combinations exist
    # Horizontal length of all possible actions
    h = comb(n,k) * len_kops
    return h


# maximizing polarization only: \\bar{z}^T \\bar{z}
def obj_polarization(A, L, op, n):
    op_mean = mean_center(op, n)
    z_mean = np.dot(A, op_mean)
    return np.dot(np.transpose(z_mean), z_mean)[0,0]

# def obj_polarization_1(A, L, op, n):  #z_mean is the same as s_mean - according to Stanford paper theory
#     z = np.dot(A, op)
#     z_mean = mean
#     return np.dot(np.transpose(z_mean), z_mean)[0,0]

# Calculate innate polarization
def obj_innate_polarization(s, n):
#     np.set_printoptions(precision=5)
    op_mean = mean_center(s, n)
    return np.dot(np.transpose(op_mean), op_mean)[0,0]

np.set_printoptions(precision=5)

In [3]:
# k = 3
# n = 20
# # #comb(n,k)
# h = len_actions(k, n)
# print(h)
def cgen(i,n,k):
    """
    returns the i-th combination of k numbers chosen from 1,2,...,n
    """
    c = []
    r = i+0
    j = -1
    for s in range(1,k+1):
        j = j+1
        while r-comb(n-1-j,k-s)>=0:
            r -= comb(n-1-j,k-s)
            j += 1
        c.append(j)
    return c

In [4]:
# create all combination
def cgen(i, n, k):
    """
    returns the i-th combination of k numbers chosen from 1,2,...,n
    """
    c = []
    r = i+0
    j = -1
    for s in range(1, k+1):
        cs = j+1
        while r-comb(n-1-cs, k-s) >= 0:
            r -= comb(n-1-cs, k-s)
            cs += 1
        c.append(cs)
        j = cs
    return c


# input representative k_nodes, nodes needs to be excluded, return the real k nodes that are available
def convert_available(k_nodes, touched, k):
    # touched = list(set(touched)) #[2,4,6,8] unqie values of touched
    touched.sort()
    for i in touched:
        for j in range(k):  # [2,3,4,5,6,7,8,9]
            if k_nodes[j] >= i:
                k_nodes[j] = k_nodes[j] + 1
    return k_nodes

# create available combination of K nodes


def creat_available_comb(i_th, n, k, touched):
    a = len(set(touched))  # number of unqiue touched nodes
    # len_nodesets = comb(n-a,k) #  number of available combination of k nodes

    k_fake = cgen(i_th, n-a, k)  # generate the i-th list from total n-a agents
    # convert the i-th list to real k nodes
    k_nodes = convert_available(k_fake, touched, k)
    print(
        f'DEBUG2 k_fake={k_fake} k_nodes={k_nodes} touched={touched} n={n} k={k} a={a} i_th={i_th}')
    return k_nodes

# def creat_all_comb(i_th, n, k):
# ########### create all combination of K opinions
#     max_option = [0, 1]
#     k_opinions =[]
#     k_opinions = product(max_option, repeat=k)  # - all k opinion combinations

#     k_nodes = cgen(i_th,n,k)

#     return(k_nodes, k_opinions)


def creat_all_comb(n, k):
    # create all combination of K opinions
    max_option = [0, 1]
    k_opinions = []
    k_opinions = product(max_option, repeat=k)  # - all k opinion combinations
    return list(k_opinions)


# node_set - 1 set  k_opinion- 1 set
def change_k_innate_opinion(s, node_set, k_opinion):
    op = copy.copy(s)  # make a copy of the innate opinion array

    for j in range(k):
        b = node_set[j]  # b - agent index
        op[b] = k_opinion[j]   # j - index of which opinion combination

    return op

### 1. Import Network

In [5]:
import numpy as np

def create_erdos_renyi_network(n, p, u, v):
    A = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
#         for j in range(i+1,n):
#             r = np.random.rand()
#             if r < p:
            if j<i:
                A[i,j] = A[j,i] = 1
#             if i==u or i==v:
#                 A[i,j] = A[j,i] = 1
    return A


# Create a synthetic Erdős-Rényi network with 10 nodes and edge probability 0.3
# p = 0.3
# u=1
# v=2

# print(s)
# # Print the adjacency matrix
# G = create_erdos_renyi_network(n, p, u, v)
# print(G)

def make_innat_opinions(n): # Make opinion for agents only - no info source is involved

    # Make list of ind innate opinion to define info source opinion
    innat_s = np.random.uniform(low=0, high=1, size=int(n))   #individual's innate opinion

    s = np.zeros((n, 1))

    idx1 = 0
    for i in range(len(s)):
        s[i] = innat_s[idx1]  #set innate opinion for ind.
        idx1 += 1
    return s

In [6]:
# # #################### Import Sythetic Network Data
df = pd.read_csv (save_path+'/data/5Adj_matrix.csv', header=None)
G = np.array(df[df.columns[:]])
# # # print(G)
df1 = pd.read_csv(save_path+'/data/5innate_op.csv', header = None)
s = np.array(df1[df1.columns[:]])

# print(s)
# print(G.shape)

# # Set n according to the data
n = len(s[:])
# # print(n)

In [7]:
import networkx as nx
import matplotlib.pyplot as plt

#n = 20 # Number of nodes

# # Create empty graph
# nxG = nx.Graph()

# # Add nodes to graph
# nxG.add_nodes_from(range(n))

# # Add edges based on node index
# for i in range(n):
#     for j in range(i):
#         p = (n-i)/(n-j) # Probability of edge existing
#         if np.random.random() < p:
#             nxG.add_edge(i, j)

# # Draw graph
# nx.draw(nxG, with_labels=True)
# plt.show()
# G = nx.adjacency_matrix(nxG).todense()
# print(G)


In [8]:
################## Process the Network Data
L = scipy.sparse.csgraph.laplacian(G, normed=False)  # Return the Laplacian matrix
A = np.linalg.inv(np.identity(n) + L)  # A = (I + L)^(-1)\n  Stanford paper theory
m = num_edges(L, n)                    # call the function to calculate the number of edges
# what the twitter graph looks like
nxG = nx.from_numpy_array(G)
#plt.figure(figsize=(20, 20))
# nx.draw(nxG).
columnsum_ij = np.sum(A, axis=0)
print(columnsum_ij)
# s = [0.5]*n
# s = np.reshape(s, (n, 1))
print(s)

[1. 1. 1. 1. 1.]
[[0.23068]
 [0.8871 ]
 [0.64121]
 [0.21768]
 [0.16186]]


In [9]:
# what the twitter graph looks like
s_use = s.flatten()   # Convert array to a list for later operation
s_use = s_use.tolist()
new_s = [i * 30 for i in s_use]
df = pd.DataFrame(new_s, columns=['Opinion']) #create a datafram with index at column 1, opinion at column 2
#_______________________________________________________

nxG = nx.from_numpy_array(G)
# nxG = nx.relabel_nodes(nxG, mapping)
plt.figure(figsize=(20, 20))


def node_edge(G, n):
    edges =[]
    for v in range(n):
        a = np.array(np.nonzero(G[v])[0])
        edge = len(a)
#         print(edge)
        edges.append(edge)

    return edges

node_edges = node_edge(G, n)
# print(node_edges)

node_sizes =[]
for i in node_edges:
    node_size = 1/i*20000
    node_sizes.append(node_size)

min_equi = []
max_equi = []
# Create node color
color_map = []
for node in nxG:
    if node in min_equi:
        color_map.append('orange')
    elif node in max_equi:
        color_map.append('red')
    else:
        color_map.append('grey')
# print(node_sizes)
#-_______________________________________
# what the twitter graph looks like
nxG = nx.from_numpy_array(G)
plt.figure(figsize=(20, 20))

# # Fix seed - fix network shape
# my_pos = nx.spring_layout(nxG, seed = 2)
# nx.draw(nxG, pos= my_pos, with_labels= True, node_color=df['Opinion'].astype(int),cmap=plt.cm.Blues, node_size= node_sizes, edge_color='black', width=0.8, font_color='black',font_size=26, font_weight='bold', alpha=0.8)
# #nx.draw(nxG, pos = my_pos, with_labels=False, node_color=color_map, node_size= node_sizes, edge_color='grey', width=0.5, font_color='white',font_size=9, font_weight='bold')
# sm = plt.cm.ScalarMappable(cmap=plt.cm.Blues, norm=plt.Normalize(vmin = 0, vmax=1))
# cbar = plt.colorbar(sm, shrink = 0.5)
# tick_font_size = 24
# cbar.ax.tick_params(labelsize=tick_font_size)
# plt.show()


<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

<Figure size 2000x2000 with 0 Axes>

In [10]:
print(s)

[[0.23068]
 [0.8871 ]
 [0.64121]
 [0.21768]
 [0.16186]]


### 2. Equilibrium & Polarization 
$$P(z) = z ^T * z $$


In [11]:
# op = s
# y = mean_center(s,n)
# # print(y)
# innat_pol = np.dot(np.transpose(y), y)[0,0]
# print('Innate_polarization:')
# print(innat_pol)

# # Test equilibrium polarization
# equ_pol = obj_polarization(A, L, s, n)
# print('Equi_polarization:')
# print(equ_pol)

# di = equ_pol-innat_pol
# print("Difference:")
# print(di)

### 3. Helper Functions

In [12]:
def k_random_play(s,n,k):  # player randomly choose an agent and randomly change the agent

    op = copy.copy(s)

    ########### create all combination of K nodes
    k_opinions = creat_all_comb(n, k)
    len_nodesets = comb(n,k)

    i_th = random.randint(0,len_nodesets-1)  # randomly select an agent index
#     v_list = node_sets[v_index]
    v_list = cgen(i_th,n,k)

    ########### create all combination of K opinions
    len_kops = len(k_opinions) # - number of combinations exist
    op_index = random.randint(0,len_kops-1) # randomly select index for an OPINION list
    new_op = k_opinions[op_index]  # randomly select an opininon list(0 and 1) to update the opinion array
    print('Nodes, opinions')
    print(v_list,new_op)
#     print(new_op)
    op = change_k_innate_opinion(s, v_list, new_op)

    #print("    "+"Agent" + str(v) +" 's opinion " + str(old_opinion) + " changed to "+ str(new_op))

    por = obj_polarization(A, L, op, n)

    column = len_kops*i_th + op_index
#     print("Network reaches stead_state Polarization: " + str(por))

    return (v_list, new_op, por, column)

In [13]:
def k_random_play_1(s,n,k,max_touched):  # player randomly choose an agent and randomly change the agent

    op = copy.copy(s)
#     max_opi_option = random.uniform(0, 1)   # options that maximizer have

    a = len(set(max_touched))  # number of unqiue touched nodes
    len_nodesets = comb(n-a,k) #  number of available combination of k nodes

    i_th = random.randint(0,len_nodesets-1)  # randomly select an action index
    v_list = creat_available_comb(i_th, n,k,max_touched)

    new_op_list = []
    for i in range(k):
        #new_op = random.uniform(0, 1)  # randomly select an opininon between 0 and 1
        new_op = 0.5
        new_op_list.append(new_op)

    new_op_list = tuple(new_op_list)
    print('Nodes, opinions')
    print(v_list,new_op_list)
    op = change_k_innate_opinion(s, v_list, new_op_list)
   # print("    "+"Agent" + str(v_list) +" 's opinion " + str(old_opinion) + " changed to "+ str(new_op))
    por = obj_polarization(A, L, op, n)
#     print("Network reaches steady-state Polarization: " + str(por))
#     print('Should be restored')
#     print(op)
    return (v_list, new_op_list, por)

### Mixed Strategy Payoff


In [14]:
np.set_printoptions(precision=3)
def make_k_payoff_row(op1, h, v2): #op1 here is only changed by Min
    payoff_row = np.zeros(h)

    column = 0
    i = 1
    for i in range(0, comb(n,k)):# i - which set of nodes option
#         print('nodeset'+ str(i))
        nodes = cgen(i,n,k)
#         print(nodes)
        k_opinions = creat_all_comb(n, k)

        for ops in k_opinions: # tuple index - select one combinatio of opinions
#             print('opset'+ str(j))
#             j = j + 1
            op2 = change_k_innate_opinion(op1, nodes, ops) # op2 has changed by both min and max now
            check =  any(node in nodes for node in v2)

 # when v1 == v2, the polarization should be negative for max, infinet for min.
 # Replace the the column_index of agent v2 with 0 for max
            if check is False:    #if v1 != v2
                # calculate the payofflarization
                payoff = obj_polarization(A, L, op2, n) # calculate the payofflarization
                payoff_row[column] = payoff
                column = column + 1
#                 print(payoff_row)
            else:                #if v1 == v2
                payoff_row[column] = 10000  # use to avoid min and max choose the same agent
                column = column + 1
#                 print(payoff_row)

#     print(payoff_row)


    return payoff_row

### Minimizer Mixed Strategy

In [15]:
# Calculate polarization of minimizer's Mixed Strategy
def mixed_K_min_polarization(s,v2,k_opinion,fla_max_fre):

    op1 = change_k_innate_opinion(s, v2, k_opinion) # only updated by minimizer's current change
    # calculate the polarization with both min(did above) and max's action(in make_payoff_row)
    payoff_row = make_k_payoff_row(op1, h, v2)  # the vector list out 2*n payoffs after min's action combine with 2*n possible max's actions
#     print('payoff_row')
#     print(payoff_row)
    #calculate fictitious payoff - equi_min
    payoff_cal = payoff_row * fla_max_fre # fla_max_fre recorded the frequency of each maximizer's action, frequency sum = 1
                                             # payoff (2*n array) * maximizer_action_frequency (2*n array)
#     print('payoff_cal')
#     print(payoff_cal)
    mixed_pol = np.sum(payoff_cal) # add up all, calculate average/expected payoff
#     print('Mixed_pol')
#     print(mixed_pol)

    # Replace the the column_index of agent v2 with -100 for max

    payoff_row = [-10000 if ele == 10000 else ele for ele in payoff_row]


    return (mixed_pol,payoff_row)


In [16]:
# Create the sum_term - exclude selected nodes
# M_1= sum_term - term_out    # numerator
def sum_rest(n, op, v2):
    # Reshape opinion array
    op = np.reshape(op, (n, 1))

    E_new = np.array([1/n] * n * n)
    # create a n*n matrix with all elements 1
    E_new = np.reshape(E_new, (n, n))
    # A_new = np.reshape(A, (n,n))
    A_new = copy.copy(A)
    A_temp = A_new-E_new
    M_new_temp = A_temp@op

    def sumFunction(x):
        s_i = op[x]*A_temp[x]
        return s_i
    np.sum(A, axis=0)
    Out_term = np.sum([sumFunction(x) for x in v2], axis=0)
    Out_term = np.reshape(Out_term, (n, 1))
    M_rest = np.transpose(M_new_temp-Out_term)
    print('M')
    print(M_rest)
    return M_rest

############## Derivate min_opinions - using above result#####################################
def k_derivate_s(v2, k, n, M):
    # k - # of selected nodes
    # V2 - selection list(k nodes) of minimier
    #  take the node index from selection list
    #  it's also the column index for these two nodes
    # op - n*1 innate opinion array that updated by maximizer
    # A - n*n adjacency matrix

    # create a parameter array with all 1/n
    c = np.array([1/n] * n)
    # c = np.reshape(c, (n,1))

    # Create left side of '=' matrix
    def leftFunction(x, y):
        a_i = np.transpose(A[x]-c)@(A[y]-c)
        return [a_i]
    a = np.concatenate([leftFunction(x, y) for y in v2 for x in v2])
    a = np.reshape(a, (k, k))
    # Create right side of '=' matrix

    def rightFunction(x, M):
        Mi = np.dot(M, (A[x]-c))
        return -Mi
    b = np.concatenate([rightFunction(x, M) for x in v2])
    result = np.linalg.solve(a, b)
    return result

In [17]:
def min_k_mixed_opinion(s, n, v2, fla_max_fre):
    weight_op = 0
    weight_M = 0
    # loop for each max_action(in total 2*n)
    k_opinions = creat_all_comb(n, k)
    len_kops = len(k_opinions)

    for column in range(h):
        if fla_max_fre[column] != 0:
            if column > k:
                nodeset_index = int(column/len_kops)
                opset_index = column % len_kops
            else:
                nodeset_index = 0
                opset_index = column

            # Calculating Max's action at this column
            v1 = cgen(nodeset_index, n, k)
            max_opinion = k_opinions[opset_index]

            # change innate opinion by max action
            op1 = change_k_innate_opinion(s, v1, max_opinion)

            # Derivate optimal Min's opinion for nodeset v2
            # {sum}{j}(s_j(h_j -c))  - rest of terms
            print(f'DEBUG0 op1 {op1} v2 {v2}')
            M_rest = sum_rest(n, op1, v2)

            # {sum}{v} p_v * M
            weight_M += fla_max_fre[column]*M_rest

    # Got optimal Min's opinion for v2
    # give a set of k weighted opinions
    k_opinion = k_derivate_s(v2, k, n, weight_M)
    # check if the result is matching hand derivative or not.
    # Game 1 results should be the same because weight it's pure strategy, same result as "check_"
    # print('result')
    # print(k_opinion)
    # #%run Check_Derivation_of_Two_Opinions.ipynb
    # (x,y) = py_pack(A, op1, n, v2)
    # (si,sl) = deriv_sty(A, op1, n, v2)

    # print('Weighted polarization')
    # print(mixed_por)

    # When the min_op is out of range(0,1), replace it with the boundary that close to min_op
    if any(x < 0 for x in k_opinion):
        # min_opinion should be in the range (0,1)
        print('Min_opinion less than 0')
        print(k_opinion)
        print('.')
        print('.')
        print('.')
        print('.')
    elif any(x > 1 for x in k_opinion):
        print('Min_opinion more than 1')
        print(k_opinion)
        print('.')
        print('.')
        print('.')
        print('.')
        # sys.exit()
    arry = np.array([])
    for i in k_opinion:
        if i < 0:
            print(i)
            i = 0
            print(i)
            arry = np.append(arry, i)
            print(arry)
        elif i > 1:
            i = 1
            arry = np.append(arry, i)
        else:
            arry = np.append(arry, i)

    (mixed_por, payoff_row) = mixed_K_min_polarization(
        s, v2, k_opinion, fla_max_fre)

    return (arry, payoff_row, mixed_por)

In [18]:
# Minimizer search: Go through each agent
def mixed_choose_min_vertex(s, n, max_touched, fla_max_fre):
    min_por = 1000 # store innate max updated polarization
    champion = (None, None, 0, None)  # assume the best action is champion

    a = len(set(max_touched))
    len_nodesets = comb(n-a, k)

    for i_th in range(len_nodesets):  #---- now v2 is a set of nodes
        v2 = creat_available_comb(i_th, n, k, max_touched)
        print(f'DEBUG 1 {v2}')
        (changed_opinion, payoff_row, por) =  min_k_mixed_opinion(s, n, v2, fla_max_fre) # find the best new_op option

        if por < min_por:  # if the recent polarization is smaller than the minimum polarization in the history
            min_por = por
            champion = (v2, changed_opinion[:], payoff_row, min_por)
            print("champion: ",champion[0],champion[1],champion[3])

            # update the recent option as champion
        # else:
        #     print('Innate polarization is smaller than Min action')

    return (champion)  # find the best minimizer's action after going through every new_op option of every agent


In [19]:
# Op has been updated by maximizer, fla_max_fre includes max's hisotry, so minimizer react to the innate op after that
def mixed_min_play(s, n, max_touched, fla_max_fre):
    min_champion = mixed_choose_min_vertex(s, n, max_touched, fla_max_fre)
    (v2, min_opinion, payoff_row, min_pol) = min_champion

    if v2 == None:    # if minimizer cannot find a action to minimize polarization after maximizer's action
        print('Minimizer fail')

    else:
        print("Minimizer finds its target agents:")
        print(v2)

        # Store innate_op of the min_selected k vertex
        old_opinion_min = [s[i] for i in v2]

        # print("    "+"Agent" + str(v2) +" 's opinion " + str(old_opinion_min) + " changed to "+ str(min_opinion))
        # print('fla_max_fre')
        # print(fla_max_fre[np.nonzero(fla_max_fre)])

        # print("Payoff row")
        # print(payoff_row)
        # print("Network reaches steady-state Polarization: " + str(min_pol))

    return (tuple(v2), payoff_row, min_opinion, min_pol)

### Maximizer Mixed Strategy

In [20]:
####Op has been updated by minimizer, fla_min_fre includes min's hisotry, so maxmizer react to the innate op after that
def k_max_polarization(payoff_matrix,column,fla_min_fre):

    # create payoff matrix for maxmizer
    payoff_vector = payoff_matrix[:,column]
    print('payoff_vector',payoff_vector [np.nonzero(payoff_vector)])
#     print(fla_min_fre [np.nonzero(fla_min_fre)])
    if any(i> 10 for i in payoff_vector) >10:
        print('Error in Payoff Matrix')
        sys.exit
    #calculate fictitious payoff - equi_max
#     print('fla_min_fre')
#     print(fla_min_fre)
    payoff_cal = payoff_vector * fla_min_fre #payoff * frequency

    mixed_pol = np.sum(payoff_cal) # add up
    #print('mixed_pol',mixed_pol)
    return mixed_pol


In [21]:
def find_idx(k_nodes, n):
    latter = 0
    index = 0
    k = len(k_nodes)

    for i in range(k):
        before = k_nodes[i] + 1
        L_min = latter + 1
        L_max = before - 1

        M = L_max - L_min

        for m in range(1, M+2):
            P = n - latter - m
            L = k - 1 - i
            index = index + comb(P, L)
        latter = before

    return index

In [22]:
# # pass on the innate opinion that has been changed by minimizer
# - op1 - innate opinion that has been changed by minimizer
def max_k_play(payoff_matrix, n, k, fla_min_fre, min_touched):
    k_opinions = creat_all_comb(n, k)
    len_kops = len(k_opinions)

    ############ start producing changes ###########
    all_por = np.zeros(h)
    print('fla_min_fre', fla_min_fre[np.nonzero(fla_min_fre)])
    a = len(set(min_touched))  # number of unique touched agent
    len_avsets = comb(n-a, k)  # length of available k_nodes combinations

    for i_th in range(len_avsets):  # for each available k nodes
        # for i in node_sets:  # for each available k nodes
        v1 = creat_available_comb(i_th, n, k, min_touched)
        # map this node set to its index located in all lists
        k_nodes_index = find_idx(v1, n)

        for f in range(len_kops):         # for each opinion combination
            # locate the column in payoff row- all combinations
            column = k_nodes_index*len_kops + f
            # por = obj_polarization(A, L, s, n)
            # calculate mixed polarization
            por = k_max_polarization(payoff_matrix, column, fla_min_fre)
            all_por[column] = por

        print('Max_por', max(all_por))

    print('all_por', all_por)
    # Index of maximum polarization - in all actions
    column = np.argmax(all_por)
    print('column - best action')
    print(column)

    (v1, max_opinion) = map_action(n, k, column)
    print(f"Maximizer found its target {k} agent: {v1} op: {max_opinion}")

    # old_opinion_max = [s[i] for i in v1]
    # check if agent's opinionis is changed or not
#     print("    "+"Agent" + str(v1) +" 's opinion " + str(old_opinion_max) + " changed to "+ str(max_opinion))
#     print("Network reaches steady-state Polarization: " + str(np.max(all_por)))

    return (v1, max_opinion, np.max(all_por), column)

## 5. Innate Op and Game

# Fictitious Play Start !


In [23]:
# op = s
# y = mean_center(s,n)
# # print(y)
# innat_pol = np.dot(np.transpose(y), y)[0,0]
# print('Innate_polarization:')
# print(innat_pol)

# # Test equilibrium polarization
# equ_pol = obj_polarization(A, L, op, n)
# print('Equi_polarization:')
# print(equ_pol)

# di = equ_pol-innat_pol
# print("Difference:")
# print(di)

In [24]:
# # Game Parameters
# Game_rounds =200 # Rounds + 1- use for printing data
# memory = 50

# Game Preparation
def push(obj, element):
    if len(obj) >= memory:
        dif = len(obj) - memory
        obj.pop(dif)
    obj.extend(list(element))
    obj = list(set(obj))

    return obj

In [25]:
def map_action(n,k,column):
    k_opinions = creat_all_comb(n, k)
    len_kops = len(k_opinions)
    nodeset_index = int(column/len_kops)
    opset_index = column%len_kops
    k_nodes = cgen(nodeset_index, n, k)
    opinions = k_opinions[opset_index]

    return (k_nodes, opinions)

In [26]:
# Game Parameters
# Game_rounds =200 # Rounds + 1- use for printing data
Game_rounds = 3
memory = 10
k = 2
experiment = 1
h = len_actions(k, n)
print(f'h={h} k={k} n={n}')

h=40 k=2 n=5


In [27]:
# def excute(h, k, Game_rounds):
# Preparation for the game
payoff_matrix = np.empty((0, h), float)
max_history = np.zeros(h, int)  # n*2 matrix, agent i & opinion options
# append a list of (agent i, min_opinion), min_opinion can be any value
min_history = []

max_history_last_100 = np.zeros(h, int)
min_history_last_100 = []

max_touched = []
min_touched = []
min_touched_all = []
min_touched_last_100 = []


# Game start from maximizer random play
#     print('Maximizer first selection')
# (v1, max_opinion, max_pol) = random_play(op,n)   # Maximizer does random action
# (v1, max_opinion, max_pol) = maximizer_fir_play(s,n,min_touched)

# (v1, max_opinion, max_pol)  = ((6,19),(1,0),0)

# (v1, max_opinion, max_pol, column) = ([0, 2], (1, 1), 0.1270803458329454, 7)
(v1, max_opinion, max_pol, column) = k_random_play(s, n, k)
First_max = (v1, max_opinion, max_pol, column)
print('First_max', First_max)


# Maximizer start with greedy play
# (v1, max_opinion, max_pol) = maximizer_fir_play(s,n,min_touched)   # Maximizer choose action greedily

max_touched.extend(tuple(v1))
# store maximizer play history, using agent(row) and changed opinion(column) as indicator to locate history
max_history[column] = max_history[column] + 1

fla_max_fre = max_history/1  # its frequency, only played  1 time so far, divided by 1

# fla_max_fre = max_frequency.flatten()   # flatten the n*2 matrix to a 2n*1 matrix
# so we can multiply the freuency (2n*1)with payoff array (1*2n)
# to get average payoff of fictitious play

# if game start from minimizer random play - make sure two random play are not same agent!!!
#     print('Minimizer first selection')
(v2, min_opinion, min_pol) = k_random_play_1(s, n, k, max_touched)
# (v2, min_opinion, min_pol) = minimizer_fir_play(s,n,min_touched)

# (v2, min_opinion, min_pol) = ([1, 3], (0.5, 0.5), 0.020147309098439665)
# (v2, min_opinion, min_pol) = ((13,15),(0.5,0.5),1)
# (v2, payoff_row, min_opinion, min_pol) = mixed_min_play(s,v1,max_opinion,n, max_touched,fla_max_fre)
First_min = (v2, min_opinion, min_pol)
print('First_min:', First_min)

if any(x in v1 for x in v2):  # if Max and Min randomly selected the same agent, then we need to restart - cannot choose same agent
    sys.exit()

# Minimizer start with greedy play
# (v2, min_opinion, min_pol) = minimizer_fir_play(s,n,max_touched)
min_touched.extend(v2)
min_touched_all.append(tuple(v2))
# store minimizer play history
#############################################################
min_history.append((tuple(v2), tuple(min_opinion)))
print(f'min_history {min_history}')
# return a dictionary include {'min_option': count of this choice}
counter = collections.Counter(min_history)
# print(counter)
# return only frequency of all min options in order
fla_min_fre = np.array(list(counter.values()))/1
# print(f'fla_min_fre {fla_min_fre}')

(_, payoff_row) = mixed_K_min_polarization(s, v2, min_opinion, fla_max_fre)
payoff_matrix = np.vstack([payoff_matrix, payoff_row])

equi_min = min_pol
equi_max = max_pol

Flag = 0
i = 0
while Flag == 0:
    i = i + 1
    print("Game " + str(i))
    print("_____________________")
    print('min_history', min_history)
    print('max_history', max_history)


#     if max_pol == min_pol:
    if i == Game_rounds:            # i == # of iterations you want to run + 2
        # because Game 101 is skipped for collecting data, to get 200 game result, we need to run 201 iteration
        print('MAX_last_100,  all')
        max_l100_fre = max_history_last_100/100
        max_fre = max_history/Game_rounds
        print(max_l100_fre[np.nonzero(max_l100_fre)],
              max_fre[np.nonzero(max_fre)])
        print(np.nonzero(max_l100_fre)[0], np.nonzero(max_fre)[0])
        columns = list(np.nonzero(max_l100_fre)[0])
        for column in list(columns):
            k_opinions = creat_all_comb(n, k)
            len_kops = len(k_opinions)
            nodeset_index = int(column/len_kops)
            opset_index = column % len_kops
            k_nodes = cgen(nodeset_index, n, k)

            opinions = k_opinions[opset_index]
            print('Max Nodes:' + str(k_nodes)+' Opinion: ' + str(opinions))

        # MINimizer's Strategy in the last 100 round
        counter = collections.Counter(min_touched_last_100)
        # return only frequency of all min options in order
        fla_min_fre = np.array(list(counter.values()))/(100)
        print('MIN_last_100,  all')
        # return a dictionary include {'min_option': count of this choice}
        counter_1 = collections.Counter(min_touched_all)
        # return only frequency of all min options in order
        fla_min_fre_1 = np.array(list(counter_1.values()))/Game_rounds
        print(fla_min_fre, fla_min_fre_1)
        print(counter, counter_1)
        print(f'Max Pol: {equi_max} Min Pol: {equi_min}')

        break

    elif equi_min == equi_max:
        print(f"Reached Nash Equilibrium at game {i} and Equi_Por = {equi_min}")
        # print(f'max_distribution {max_frequency}')
        # print(f'min_distribution {fla_min_fre}')
        Flag = 1
        break
# ________________________________________________________________
    # maximizer play
    else:
        if i == Game_rounds-100:  # if Game_round = 200, after 100 iteration, Game 101 print previous historical result

            # Remove max frequency less than 0.1--
            max_history_last_100 = np.zeros(h)
            min_history_last_100 = []
            min_touched_last_100 = []

        (v1, max_opinion, equi_max, column) = max_k_play(
            payoff_matrix, n, k, fla_min_fre, min_touched)
        max_touched = push(max_touched, v1)

        # cumulate strategy
        max_history[column] = max_history[column] + 1
        max_history_last_100[column] = max_history_last_100[column] + 1

        # max_frequency to calculate average payoff
        fla_max_fre = max_history/(i+1)
        print(f'fre_max at spot {fla_max_fre[column]}')

        # MINImizer play
        (v2, payoff_row, min_opinion, equi_min) = mixed_min_play(
            s, n, max_touched, fla_max_fre)
        min_touched = push(min_touched, v2)
        min_touched_all.append(v2)
        min_touched_last_100.append(v2)

        # print(v2, min_opinion, min_pol)
        if tuple(tuple(v2)+min_opinion) in counter.keys():
            # if this min_option is in min_history, no need to update paryoff matrix, only update frequency
            payoff_matrix = payoff_matrix
            # print("Same history")
            # print((str(v2),str(min_opinion)))
        else:
            # if this is a new option, append to previous matrix
            payoff_matrix = np.vstack([payoff_matrix, payoff_row])
            # print(f'payoff_row {payoff_row}')

        min_history.append((tuple(v2), tuple(min_opinion)))
        min_history_last_100.append(tuple(v2 + min_opinion))
        print('min_history')
        print(min_history)
        # return a dictionary include {'min_option': count of this choice}
        counter = collections.Counter(min_history)
        print(counter)
        # print('counter.keys:', counter.keys())
        # print(counter.keys())
        # return only frequency of all min options in order
        fla_min_fre = np.array(list(counter.values()))/(i+1)
        print('fla_min_fre:', fla_min_fre)
        # print(fla_min_fre)
        # print("Not Reached Nash Equilibrium at Equi_Min = " + str(equi_min) + " and Equi_Max = "+ str(equi_max))

result = (First_max, First_min, max_touched, min_touched, payoff_matrix, min_history, fla_min_fre, min_history_last_100,
          min_touched_last_100, min_touched_all, max_history, fla_max_fre, max_history_last_100, equi_max, equi_min)

#     return result

Nodes, opinions
[1, 2] (0, 1)
First_max ([1, 2], (0, 1), 0.033878134298830756, 17)
DEBUG2 k_fake=[0, 4] k_nodes=[0, 4] touched=[1, 2] n=5 k=2 a=2 i_th=1
Nodes, opinions
[0, 4] (0.5, 0.5)
First_min: ([0, 4], (0.5, 0.5), 0.012700877654037955)
min_history [((0, 4), (0.5, 0.5))]
Game 1
_____________________
min_history [((0, 4), (0.5, 0.5))]
max_history [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]
fla_min_fre [1.]
DEBUG2 k_fake=[1, 2] k_nodes=[1, 2] touched=[0, 4] n=5 k=2 a=2 i_th=0
payoff_vector [0.028]
payoff_vector [0.022]
payoff_vector [0.022]
payoff_vector [0.034]
Max_por 0.033797021050447185
DEBUG2 k_fake=[1, 3] k_nodes=[1, 3] touched=[0, 4] n=5 k=2 a=2 i_th=1
payoff_vector [0.016]
payoff_vector [0.019]
payoff_vector [0.024]
payoff_vector [0.016]
Max_por 0.033797021050447185
DEBUG2 k_fake=[2, 3] k_nodes=[2, 3] touched=[0, 4] n=5 k=2 a=2 i_th=2
payoff_vector [0.021]
payoff_vector [0.022]
payoff_vector [0.033]
payoff_vector [0.024]
Max_por 0.033797

In [28]:
# print(s)
# np.savetxt('Nt1_Innate_opinion.csv', s, delimiter=",")
# np.savetxt('Nt1_Adjacency_matrix.csv', G, delimiter=",")
# save_(result, k, experiment)

In [29]:
def save_(result, k, experiment):
    (First_max, First_min, max_touched, min_touched, payoff_matrix, min_history, fla_min_fre, min_history_last_100, min_touched_last_100, min_touched_all, max_history, fla_max_fre, max_history_last_100, equi_max, equi_min) = result
    pd.DataFrame(payoff_matrix).to_csv('Payoff Matrix'+ str(k) +'.'+str(experiment)+'.csv')

    with open('Result'+ str(k) +'.'+str(experiment)+'.txt', "a") as f:
        print('Initial Condition -(agent, opinion, pol)', file=f)
#         print('Innate op'+str(s),file=f)
#         print('Adjacency matrix'+ str(G), file=f)
#         print('Selected Nodeset, k_Opinions, Steady-state polarization',file=f)
        print('Max:'+ str(First_max), file=f)
        print('Min' + str(First_min), file=f)

        print('_____________________', file=f)
        print('Max Pol: '+str(equi_max)+"  Min Pol: "+str(equi_min))
        # MAXimizer's distribution of LAST 100 iteration
        print('Max_distribution_last_100',file = f)
        max_l100_fre = max_history_last_100/100
        print(max_l100_fre [np.nonzero(max_l100_fre)],file = f)
        # print for small network
        #print(max_history_last_100)
        # # Print for Large Network
        print(np.nonzero(max_l100_fre),file = f)
        columns = np.nonzero(max_l100_fre)
        columns = list(columns[0])
        for column in columns:
            (k_nodes, opinions) = map_action(n,k,column)
            print('  Max Nodes:'+ str(k_nodes)+' Opinion: '+ str(opinions), file = f)


        print('Max_distribution_all',file = f)
        max_fre = max_history/Game_rounds
        print(max_fre[np.nonzero(max_fre)],file = f)
        print([np.nonzero(max_fre)],file = f)
        columns_all = np.nonzero(max_l100_fre)
        columns_all = list(columns_all[0])
        for column in columns_all:
            (k_nodes, opinions) =  map_action(n,k,column)
            print('  Max Nodes:'+ str(k_nodes)+' Opinion: '+ str(opinions), file = f)


        # MINimizer's Strategy in the last 100 round
        counter=collections.Counter(min_touched_last_100)
        fla_min_fre = np.array(list(counter.values()))/(100) #return only frequency of all min options in order
        print('Min_distribution_last_100',file = f)
        print(fla_min_fre,file = f)
        print(counter,file = f)
        # print(min_touched_last_100)

        counter_1=collections.Counter(min_touched_all)  #return a dictionary include {'min_option': count of this choice}
        fla_min_fre_1 = np.array(list(counter_1.values()))/Game_rounds #return only frequency of all min options in order
        print('Min_distribution_all',file = f)
        print(fla_min_fre_1,file = f)
        print(counter_1,file = f)
        np.set_printoptions(precision=3)

        counter_a=collections.Counter(min_history)  #return a dictionary include {'min_option': count of this choice}
        print(counter_a, file=f)

        print('min_recent_'+str(memory)+'_touched', file=f)# then stop at Game 202
        print(min_touched, file=f)
        print('max_recent_'+str(memory)+'_touched', file=f)
        print(max_touched, file=f)